In [ ]:
import pandas as pd
from utils.lngselection import abbreviation
from wikiwho_wrapper import WikiWho
from external.wikipedia import WikipediaDV, WikipediaAPI
from metrics.conflict import ConflictManager
import numpy as np
import pyLDAvis
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary
from biterm.btm import oBTM

In [ ]:
%%capture
## Some Extensions ##
%load_ext autoreload
%autoreload 2
%store -r the_page

if 'the_page' not in locals():
    import pickle
    print("Loading default data...")
    the_page = pickle.load(open("data/the_page.p",'rb'))

lng = abbreviation('English')

In [ ]:
wikiwho = WikiWho(lng=lng)
all_content = wikiwho.dv.all_content(the_page['page_id'])
revisions = wikiwho.dv.rev_ids_of_article(the_page['page_id'])

In [ ]:
con_manager = ConflictManager(all_content.copy(), 
                                           revisions.copy(), 
                                           lng=lng, 
                                           include_stopwords=False)

In [ ]:
con_manager.calculate()
token = con_manager.all_actions.copy()
tokens_processed = token[['rev_id', 'rev_time', 'editor', 'token_id', 'token']].groupby("rev_id")['token_id'].apply(lambda group_series: group_series.to_numpy()).reset_index()
tokens_processed['token_id']

In [ ]:


def dummy(doc):
    return doc

vec = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
    )  

for i, row in tokens_processed[np.array(list(map(len,tokens_processed.token_id.values)))==1].iterrows():
    k = random.choice([-1, 1])
    np.append(tokens_processed.loc[i+k, 'token_id'], row['token_id'][0])
tokens_processed = tokens_processed[np.array(list(map(len,tokens_processed.token_id.values)))>1]
token_ids = token[['token', 'token_id']].drop_duplicates()['token_id'].to_numpy()
#tokens_processed.drop(443, inplace=True)
#tokens_processed.reset_index(inplace=True)
#X_old = vec.fit_transform(tokens_processed['token_id'].tolist()).toarray()
X = tokens_processed['token_id'].to_numpy()
X_new = np.empty([1, 4753], dtype=int)
for x in X:
    X_new = np.append(X_new, [[1 if i in x else 0 for i in token_ids]], axis=0)
    
X_new = np.delete(X_new, 0, 0)

In [ ]:
#vocab = np.array(vec.get_feature_names())
vocab = token[['token', 'token_id']].drop_duplicates()['token'].to_numpy()
biterms = vec_to_biterms(X_new)

In [ ]:
btm = oBTM(num_topics=10, V=vocab) #default alpha=1., beta=0.01, l=0.5
topics = btm.fit_transform(biterms, iterations=100)

In [ ]:
vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X_new, axis=1), vocab, np.sum(X_new, axis=0))

pyLDAvis.save_html(vis, 'topic_modelling_26_06.html')  # path to output

#print("\n\n Topic coherence ..")
#topic_summuary(btm.phi_wz.T, X, vocab, 10)


In [ ]:
pyLDAvis.display(vis)